In [13]:
import pandas as pd
import numpy as np

# Forward

In [8]:
def forward(seq, PI, P, E):  # P is our Transition matrix && E is our Emission matrix
    N = len(PI) 
    T = len(seq) # Time

    alpha = np.zeros((N, T))
    alpha[:, 0] = PI * E[:, seq[0]]

    for t in range(1, T):
        for j in range(N):
            alpha[j, t] = np.sum(alpha[:, t-1] * P[:, j]) * E[j, seq[t]]

    return alpha

In [9]:
seq = np.array([0, 1, 0])
P = np.array([[0.99,0.01],[0.01,0.99]]) 
E = np.array([[0.8,0.2],[0.1,0.9]])
PI = np.array([0.99, 0.01])

alpha_values = forward(seq, PI, P, E)

# Print results
print("Forward Probabilities:")
print(alpha_values)

Forward Probabilities:
[[0.792      0.156818   0.12426401]
 [0.001      0.008019   0.0009507 ]]


# Backword

In [10]:
def backward(seq, P, E):
    N = P.shape[0]
    T = len(seq)

    beta = np.zeros((N, T))
    beta[:, -1] = 1

    for t in range(T - 2, -1, -1):
        for i in range(N):
            beta[i, t] = np.sum(beta[:, t + 1] * P[i, :] * E[:, seq[t + 1]])

    return beta

In [11]:
seq = np.array([0, 1, 0])
P = np.array([[0.99,0.01],[0.01,0.99]]) 
E = np.array([[0.8,0.2],[0.1,0.9]])

beta_values = backward(seq, P, E)

# Print results
print("Backward Probabilities:")
print(beta_values)

Backward Probabilities:
[[0.157977 0.793    1.      ]
 [0.096923 0.107    1.      ]]


# Baum_Welch with input from user 

In [6]:
# Forward function
def forward(seq, PI, P, E):
    N = len(PI)
    T = len(seq)

    alpha = np.zeros((N, T))
    alpha[:, 0] = PI * E[:, seq[0]]

    for t in range(1, T):
        for j in range(N):
            alpha[j, t] = np.sum(alpha[:, t-1] * P[:, j]) * E[j, seq[t]]

    return alpha
#=========================================================================

# Backward function
def backward(seq, P, E):
    N = P.shape[0]
    T = len(seq)

    beta = np.zeros((N, T))
    beta[:, -1] = 1

    for t in range(T - 2, -1, -1):
        for i in range(N):
            beta[i, t] = np.sum(beta[:, t + 1] * P[i, :] * E[:, seq[t + 1]])

    return beta
#=========================================================================

# Baum-Welch function
def baum_welch(seq, N, M, max_iters=100, tol=1e-4):
    T = len(seq)

    PI = np.round(np.random.rand(N), 2)
    P = np.round(np.random.rand(N, N), 2)
    E = np.round(np.random.rand(N, M), 2)

    for iter in range(max_iters):
        # E-step
        alpha = forward(seq, PI, P, E)
        beta = backward(seq, P, E)
        xi = np.zeros((N, N, T-1))

        for t in range(T-1):
            for i in range(N):
                for j in range(N):
                    xi[i, j, t] = (alpha[i, t] * P[i, j] * E[j, seq[t+1]] * beta[j, t+1]) / np.sum(alpha[:, t] * beta[:, t])

        # M-step
        PI = alpha[:, 0] / np.sum(alpha[:, 0])

        for i in range(N):
            for j in range(N):
                P[i, j] = np.sum(xi[i, j, :]) / np.sum(np.sum(xi[i, :, :]))

        for j in range(N):
            for k in range(M):
                E[j, k] = np.sum(alpha[j, seq == k]) / np.sum(alpha[j, :])

        # Check for convergence
        if np.abs(np.log(np.sum(alpha[:, -1]))) < tol:
            break

    return PI, P, E
#=========================================================================

# Taking input from the user
seq_str = input("Enter the sequence (comma-separated values): ")
seq = np.array([int(x) for x in seq_str.split(',')])

N = int(input("Enter the number of hidden states (N): "))
M = int(input("Enter the number of events (M): "))
#=========================================================================

# Using PI_hat, P_hat, E_hat
PI_hat, P_hat, E_hat = baum_welch(seq, N, M)
print('Estimated "PI":')
print(PI_hat)
print('\nEstimated "P" Matrix:')
print(P_hat)
print('\nEstimated "E" Matrix:')
print(E_hat)
#=========================================================================

# Using pi_hat, A_hat, B_hat
pi_hat, A_hat, B_hat = baum_welch(seq, N, M)
print('Estimated "PI":')
print(np.round(pi_hat, decimals=4))
print('\nEstimated "P" Matrix:')
print(np.round(A_hat, decimals=4))
print('\nEstimated "E" Matrix:')
print(np.round(B_hat, decimals=4))

Enter the sequence (comma-separated values): 0,1,0
Enter the number of hidden states (N): 2
Enter the number of events (M): 2
Estimated "PI":
[1.00000000e+00 5.01297672e-12]

Estimated "P" Matrix:
[[9.79084070e-78 1.00000000e+00]
 [1.00000000e+00 4.18704158e-11]]

Estimated "E" Matrix:
[[1.00000000e+00 1.40101057e-32]
 [4.68831398e-11 1.00000000e+00]]
Estimated "PI":
[1. 0.]

Estimated "P" Matrix:
[[0. 1.]
 [1. 0.]]

Estimated "E" Matrix:
[[1. 0.]
 [0. 1.]]
